# Serving Predictions on Models developed by SQL ML Approach

The models developed previously in section (Machine Learning with SQL) can be exported and deployed through enpoints for prediction serving. This implmentation deals with the process of exporting and deploying the model through endpoints.

## Setting Up

In [1]:
REGION = 'us-central1'
PROJECT_ID='mlopstqc'
DATANAME = 'leaktest'
IDENTIFIER = '5'

# Resources
DEPLOY_IMAGE='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-3:latest'
DEPLOY_COMPUTE = 'n1-standard-2'

# Model Training
VAR_TARGET = 'TestPressureMbar'
VAR_OMIT = 'transaction_id' 

In [2]:
from google.cloud import aiplatform
from datetime import datetime

from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

In [3]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bigquery = bigquery.Client()

In [5]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{DATANAME}/models/{IDENTIFIER}"
params = {"URI": URI}
DIR = f"temp/{IDENTIFIER}"

In [23]:
!rm -rf {DIR}
!mkdir -p {DIR}

E0405 13:41:15.004532915   14575 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0405 13:41:15.179704747   14575 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


## Exporting the Model (ML with SQL)

In [6]:
export = bigquery.query(query = f"EXPORT MODEL {DATANAME}.{DATANAME}_lr OPTIONS(URI = '{URI}')")


In [7]:
export.result()

## Serving the Model

### Uploading the Model Exported

In [8]:
model = aiplatform.Model.upload(
    display_name = f'{IDENTIFIER}_{DATANAME}_{TIMESTAMP}',
    serving_container_image_uri = DEPLOY_IMAGE,
    artifact_uri = URI,
    labels = {'identifier':f'{IDENTIFIER}'}
)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/969346734638/locations/us-central1/models/8427242055461240832/operations/486822585812647936
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/969346734638/locations/us-central1/models/8427242055461240832
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/969346734638/locations/us-central1/models/8427242055461240832')


In [9]:
model.display_name

'5_leaktest_20220405131347'

### Creating the Endpoint

In [10]:
endpoint = aiplatform.Endpoint.create(
    display_name = f'{IDENTIFIER}_{DATANAME}_{TIMESTAMP}',
    labels = {'identifier':f'{IDENTIFIER}'}
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/969346734638/locations/us-central1/endpoints/8913300961728921600/operations/549872980595834880
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/969346734638/locations/us-central1/endpoints/8913300961728921600
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/969346734638/locations/us-central1/endpoints/8913300961728921600')


## Predicting from Endpoint

### Preparing the record for prediction

In [12]:
pred = bigquery.query(query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' LIMIT 10").to_dataframe()


In [13]:
pred.head(4)

,TestVolumeLitre,AlarmDiffPressure,TestPressureMbar,transaction_id,splits
0,0.2,500,12800,68cfe17b-33f6-4240-9254-d77514feb083,TEST
1,10.0,320,12800,e6cc26be-a050-4c91-99cf-52c2dcea0924,TEST
2,0.1,320,12800,b5e82b71-f62a-400a-9cf1-f551b2b57bf9,TEST
3,0.1,320,12800,d2f9cc2b-4aed-4734-a723-17b44020021b,TEST


In [14]:
newob = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET,'splits'])]].to_dict(orient='records')[0]


In [15]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

In [16]:
instances

[struct_value {
   fields {
     key: "AlarmDiffPressure"
     value {
       number_value: 500.0
     }
   }
   fields {
     key: "TestVolumeLitre"
     value {
       number_value: 0.2
     }
   }
 }]

### Python Client Predictions

In [20]:
prediction = endpoint.predict(instances=instances, parameters=parameters)
prediction

FailedPrecondition: 400 Endpoint projects/969346734638/locations/us-central1/endpoints/8913300961728921600 doesn't have traffic_split.

In [21]:
prediction.predictions[0]

NameError: name 'prediction' is not defined

In [ ]:
prediction.predictions[0][f'{VAR_TARGET}_values'][np.argmax(prediction.predictions[0][f'{VAR_TARGET}_probs'])]


### REST Predictions

In [24]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [25]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

E0405 13:41:42.767543540   14575 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


{
  "error": {
    "code": 400,
    "message": "Endpoint projects/969346734638/locations/us-central1/endpoints/8913300961728921600 doesn't have traffic_split.",
    "status": "FAILED_PRECONDITION"
  }
}


### Terminal

In [26]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json


E0405 13:43:24.345994579   14575 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
ERROR: (gcloud.beta.ai.endpoints.predict) FAILED_PRECONDITION: Endpoint projects/mlopstqc/locations/us-central1/endpoints/8913300961728921600 doesn't have traffic_split.
